# List of all emoticons as GIFs

(last updated 2016-02-16)

The code below generates a list of all emoticons in dota and their unicode character. You can copy & paste the character into Dota 2 to use the emoticon. It doesn't matter if you don't own the emoticon pack. This works in Reborn!

In [ ]:
from __future__ import unicode_literals
import os
from glob import glob
import vdf
import vpk
from PIL import Image
from collections import OrderedDict
from IPython.display import HTML

out_temp = "./.etmp"
out_dir = "resources/emoticons"
out_sizes = [32, 64]
emoticons_root = "resource/flash3/images/emoticons/{0:s}"

pak1 = vpk.open("/d/Steam/steamapps/common/dota 2 beta/game/dota/pak01_dir.vpk")
data = vdf.loads(pak1['scripts/emoticons.txt'].read().decode('utf-16le'), mapper=OrderedDict)

resp = "<table><tr>" + "<th colspan={}>preview</th><th>chr</th><th>alias</th>".format(len(out_sizes))*3 + "</tr>\n"

if not os.path.isdir(out_temp): os.mkdir(out_temp)

for k, v in data['emoticons'].items():   
    k = int(k)
    f = pak1.get_file(emoticons_root.format(v['image_name']))
    filename = "{0:0>3d}_{1:0>8x}".format(k, f.crc32)
    out_paths = []
    for size in out_sizes:
        out_paths.append(os.path.join(out_dir, "{0}_{1:d}.gif".format(filename, size)))

    # if a gif doesn't exist generate it
    if not os.path.isfile(out_paths[0]):   
        # remove any existing gifs for current emoticon and clear out_temp    
        for filepath in glob("{}/*".format(out_temp)) + glob(os.path.join(out_dir, "{0:0>3d}_*".format(k))):
            os.remove(filepath)

        # split the png sequence in seperate files                              
        im = Image.open(f)
        for out_path, size in zip(out_paths, out_sizes):
            for i in range(im.size[0] / 32):
                im.crop((i*32, 0, (i+1)*32, 32))\
                .resize((size, size), Image.NEAREST)\
                .save("{0}/{1:0>3d}.png".format(out_temp, i))
            # combines the sequence images into a gif using imagemagik
            !convert -loop 0 -delay 10 -alpha set -dispose previous "$out_temp/*.png" "$out_path"
    
    # generate table cells for emoticon
    if (int(k) - 1) % 3 == 0:
        resp += "<tr>\n"    
    for path in out_paths:
        resp += "<td><img src='{}'></td>\n".format(path)
    resp += "<td>{0:s}</td>\n<td>:{1:s}:</td>\n".format(
            unichr(0xE000 + int(k)),       
            v['aliases']['0'],
            )
    if (int(k) - 1) % 3 == 2:
        resp += "</tr>\n"

!rm -rf "$out_temp"
if (int(k) - 1) % 3 != 2:
    resp += "</tr>"
resp += "</table>"

# Emoticon table

Copy the character (little sqare) from the `chr` column and paste it in chat

In [3]:
HTML(resp)